# MAX OVER

In [1]:
import findspark
findspark.init()

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/28 15:36:57 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
data_product = [
    (1,"mobile",1000, 10, 7),
    (1,"mobile",1400, 5, 8),
    (1,"mobile",1200, 7, 10),
    (2,"laptop",2250, 2, 27),
    (2,"laptop",3150, 14, 29),
    (2,"laptop",3000, 8, 25),
    (2,"laptop",4500, 2, 27)
]

schema_product = "id INTEGER, device STRING, price INTEGER, discount INTEGER, size INTEGER"

In [4]:
df_product = spark.createDataFrame(data = data_product, schema = schema_product)
df_product.show()

+---+------+-----+--------+----+
| id|device|price|discount|size|
+---+------+-----+--------+----+
|  1|mobile| 1000|      10|   7|
|  1|mobile| 1400|       5|   8|
|  1|mobile| 1200|       7|  10|
|  2|laptop| 2250|       2|  27|
|  2|laptop| 3150|      14|  29|
|  2|laptop| 3000|       8|  25|
|  2|laptop| 4500|       2|  27|
+---+------+-----+--------+----+



In [5]:
from pyspark.sql import Window
from pyspark.sql.functions import max, col

In [6]:
window_definition = Window.partitionBy("id")

In [7]:
df_max_values = df_product.withColumn("max_price", max("price").over(window_definition)) \
                            .withColumn("max_discount", max("discount").over(window_definition)) \
                            .withColumn("max_size", max("size").over(window_definition))
df_max_values.show()

+---+------+-----+--------+----+---------+------------+--------+
| id|device|price|discount|size|max_price|max_discount|max_size|
+---+------+-----+--------+----+---------+------------+--------+
|  1|mobile| 1000|      10|   7|     1400|          10|      10|
|  1|mobile| 1400|       5|   8|     1400|          10|      10|
|  1|mobile| 1200|       7|  10|     1400|          10|      10|
|  2|laptop| 2250|       2|  27|     4500|          14|      29|
|  2|laptop| 3150|      14|  29|     4500|          14|      29|
|  2|laptop| 3000|       8|  25|     4500|          14|      29|
|  2|laptop| 4500|       2|  27|     4500|          14|      29|
+---+------+-----+--------+----+---------+------------+--------+



In [8]:
df_max_values.select(col("id"),
                     col("device"),
                     col("max_price").alias("price"),
                     col("max_discount").alias("discount"),
                     col("max_size").alias("size")).show()

+---+------+-----+--------+----+
| id|device|price|discount|size|
+---+------+-----+--------+----+
|  1|mobile| 1400|      10|  10|
|  1|mobile| 1400|      10|  10|
|  1|mobile| 1400|      10|  10|
|  2|laptop| 4500|      14|  29|
|  2|laptop| 4500|      14|  29|
|  2|laptop| 4500|      14|  29|
|  2|laptop| 4500|      14|  29|
+---+------+-----+--------+----+



In [9]:
df_max_values.select(col("id"),
                     col("device"),
                     col("max_price").alias("price"),
                     col("max_discount").alias("discount"),
                     col("max_size").alias("size")).dropDuplicates().show()

+---+------+-----+--------+----+
| id|device|price|discount|size|
+---+------+-----+--------+----+
|  1|mobile| 1400|      10|  10|
|  2|laptop| 4500|      14|  29|
+---+------+-----+--------+----+

